In [2]:
import json
from all_def import DomClickApi
import requests
import pandas as pd
from multiprocessing import *
from bs4 import BeautifulSoup
from queue import Queue
import traceback
from threading import Thread
from datetime import datetime
import numpy as np

In [3]:
dca = DomClickApi()

In [5]:
df = pd.read_csv("pc2.csv").drop(labels=["id"], axis=1)
df

,address,price,floor,total_floor,rooms,square,city,remont,balcon,uid,view,year,type_perec,hot_water,fundament,kitchen
0,"бульвар Гусева, 57",4150000,7,14,1,38.7,Тверь,standard,1,2058968711,garden,NaN,NaN,NaN,NaN,NaN
1,"улица Благоева, 3к3",3320000,1,5,1,33.8,Тверь,standard,1,2058538048,garden,NaN,NaN,NaN,NaN,NaN
2,"улица Хромова, 14",2850000,3,3,1,31.0,Тверь,standard,1,1795881805,garden,NaN,NaN,NaN,NaN,NaN
3,"улица Коробкова, 17",3200000,2,5,1,34.0,Тверь,standard,1,2059250631,garden,NaN,NaN,NaN,NaN,NaN
4,"улица С.Я. Лемешева, 10",4450000,7,16,1,37.7,Тверь,standard,1,2056021230,garden,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15485,"Кемская улица, 14",66500000,2,7,4,124.7,Санкт-Петербург,design,2,1633885501,street,NaN,NaN,NaN,NaN,NaN
15486,"Парадная улица, 3к2",79000000,6,8,4,153.0,Санкт-Петербург,well_done,2,2056289102,street,NaN,NaN,NaN,NaN,NaN
15487,"Парадная улица, 3к2",79000000,6,8,4,153.0,Санкт-Петербург,well_done,2,2056289102,street,NaN,NaN,NaN,NaN,NaN
15488,"Парадная улица, 3к2",79000000,6,8,4,153.0,Санкт-Петербург,well_done,2,2056289102,street,NaN,NaN,NaN,NaN,NaN


In [6]:
def get_extra_data(ui):
    dca = DomClickApi()
    try:
        res = dca.get(f"https://domclick.ru/card/sale__flat__{ui}", params={})
        soup = BeautifulSoup(res.content, 'html.parser')
       
        p1 = soup.find('li', {'data-e2e-id': 'Год постройки'})
        year = p1.find('span', {'data-e2e-id': 'Значение'}).text if p1 else 0 

        p2 = soup.find('li', {'data-e2e-id': 'Тип фундамента'})
        fundament = p2.find('span', {'data-e2e-id': 'Значение'}).text if p2 else 0

        p3 = soup.find('li', {'data-e2e-id': 'Тип перекрытий'})
        type_perec = p3.find('span', {'data-e2e-id': 'Значение'}).text if p3 else 0

        p4 = soup.find('li', {'data-e2e-id': 'Горячее водоснабжение'})
        hot_water = p4.find('span', {'data-e2e-id': 'Значение'}).text if p4 else 0

        p5 = soup.find('li', {'data-e2e-id': 'Кухня'})
        kitchen = p5.find('span', {'data-e2e-id': 'Значение'}).text if p5 else 0
        return year, type_perec, hot_water, fundament, kitchen
    except:
        pass 


In [7]:
print(get_extra_data(2058968711))

('2022', 'Железобетонный', 'Поквартирный котел', 0, '9,8\xa0м2')


In [9]:
list_of_dataframes = np.array_split(df, 5)


In [10]:
def extra_data(i):
        for index, row in list_of_dataframes[i].iterrows():
            uid = row['uid']
            b, y, s, u, t = get_extra_data(uid)
            list_of_dataframes[i].loc[index, 'year'] = b
            list_of_dataframes[i].loc[index, 'type_perec'] = y
            list_of_dataframes[i].loc[index, 'hot_water'] = s
            list_of_dataframes[i].loc[index, 'fundament'] = u
            list_of_dataframes[i].loc[index, 'kitchen'] = t
            list_of_dataframes[i].to_csv(f"data_{i}.csv")

In [114]:
for b in range(17):
    extra_data(b)

C:\Users\danii\AppData\Local\Temp\ipykernel_21748\406894499.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1971' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  list_of_dataframes[i].loc[index, 'year'] = b
C:\Users\danii\AppData\Local\Temp\ipykernel_21748\406894499.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6,1 м2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  list_of_dataframes[i].loc[index, 'kitchen'] = t
C:\Users\danii\AppData\Local\Temp\ipykernel_21748\406894499.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Железобетонный' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  list_of_dataframes[

In [118]:
list_of_dataframes[1].to_csv("final_data.csv")

OK


In [113]:
result
#result.to_csv("mau.csv")

,address,price,floor,total_floor,rooms,square,city,remont,balcon,uid,view,year,type_perec,hot_water,fundament,kitchen
0,"бульвар Гусева, 57",4150000,7,14,1,38.7,Тверь,standard,1,2058968711,garden,NaN,NaN,NaN,NaN,NaN
1,"улица Благоева, 3к3",3320000,1,5,1,33.8,Тверь,standard,1,2058538048,garden,NaN,NaN,NaN,NaN,NaN
2,"улица Хромова, 14",2850000,3,3,1,31.0,Тверь,standard,1,1795881805,garden,NaN,NaN,NaN,NaN,NaN
3,"улица Коробкова, 17",3200000,2,5,1,34.0,Тверь,standard,1,2059250631,garden,NaN,NaN,NaN,NaN,NaN
4,"улица С.Я. Лемешева, 10",4450000,7,16,1,37.7,Тверь,standard,1,2056021230,garden,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15482,"проспект Непокорённых, 49к2",20500000,7,16,4,135.0,Санкт-Петербург,design,2,1870651187,street,NaN,NaN,NaN,NaN,NaN
15483,"Морской проспект, 33",98900000,2,7,4,241.8,Санкт-Петербург,design,2,1525960658,street,NaN,NaN,NaN,NaN,NaN
15484,"Ушаковская набережная, 3к2",85000000,13,17,4,156.0,Санкт-Петербург,design,2,2059035935,street,NaN,NaN,NaN,NaN,NaN
15485,"Кемская улица, 14",66500000,2,7,4,124.7,Санкт-Петербург,design,2,1633885501,street,NaN,NaN,NaN,NaN,NaN
